# Transform ASK library

In [1]:
from datasets import load_dataset

dataset = load_dataset("ctu-aic/ask_library_cs", split="train")
dataset

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language', 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor'],
    num_rows: 16273
})

## Korektor

In [10]:
!/home/mlynatom/korektor/korektor-2.0.0-bin/bin-linux64/korektor

Usage: /home/mlynatom/korektor/korektor-2.0.0-bin/bin-linux64/korektor [options] model_configuration
Options: --input=untokenized|untokenized_lines|segmented|vertical|horizontal
         --output=original|xml|vertical|horizontal
         --corrections=maximum_number_of_corrections
         --viterbi_order=viterbi_decoding_order
         --viterbi_beam_size=maximum_viterbi_beam_size
         --viterbi_stage_pruning=maximum_viterbi_stage_cost_increment
         --context_free
         --version
         --help


In [ ]:
import subprocess

def call_

command = "echo 'cau jak se mas?' | /home/mlynatom/korektor/korektor-2.0.0-bin/bin-linux64/korektor /home/mlynatom/korektor/korektor-czech-130202/diacritics_h2mor.conf"

result = subprocess.run(command, shell=True, text=True, capture_output=True)

print(result.stdout.strip())

čau jak se máš?


In [14]:
#korektor
import requests
import subprocess

korektor_url = "https://lindat.mff.cuni.cz/services/korektor/api/correct"
url_model_diacritics = 'czech-diacritics_generator-130202'
url_model_spell = 'czech-spellchecker-130202'

korektor_path = "/home/mlynatom/korektor/korektor-2.0.0-bin/bin-linux64/korektor"
path_model_diacritics = "/home/mlynatom/korektor/korektor-czech-130202/diacritics_h2mor.conf"
path_model_spell = "/home/mlynatom/korektor/korektor-czech-130202/spellchecking_h2mor.conf"


def correct_text(text, model, korektor_url=None, korektor_path=None):
    if korektor_url is None and korektor_path is None:
        raise ValueError("Either korektor_url or korektor_path must be provided")
    elif korektor_url is not None:
        params = {
            'model': model,
            'data': text,
        }
        response = requests.post(korektor_url, data=params)
        if response.status_code != 200:
            raise Exception(f"Failed to correct text: {response.text}")
        corrected_text = response.json()['result']
    else:
        command = f"echo '{text}' | {korektor_path} {model}"
        result = subprocess.run(command, shell=True, text=True, capture_output=True)
        corrected_text = result.stdout.strip()
    return corrected_text



In [15]:
correct_text("Toto je testovací veta bez diakritiky.", korektor_path = korektor_path, model = path_model_diacritics)

'Toto je testovací věta bez diakritiky.'

In [16]:
def correct_map(example):
    if example['language'] == 'cs':
        example['question_diacritics_korektor'] = correct_text(example['question'],korektor_path = korektor_path, model = path_model_diacritics)
        example['answer_diacritics_korektor'] = correct_text(example['answer'],korektor_path = korektor_path, model = path_model_diacritics)
        example['question_spell_korektor'] = correct_text(example['question'], korektor_path = korektor_path, model = path_model_spell)
        example['answer_spell_korektor'] = correct_text(example['answer'], korektor_path = korektor_path, model = path_model_spell)
    else:
        example['question_diacritics_korektor'] = ""
        example['answer_diacritics_korektor'] = ""
        example['question_spell_korektor'] = ""
        example['answer_spell_korektor'] = ""
    return example

In [17]:
dataset = dataset.map(correct_map)

Map:   0%|          | 0/16273 [00:00<?, ? examples/s]

In [18]:
dataset

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language', 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor'],
    num_rows: 16273
})

In [25]:
from huggingface_hub import login
login()

In [26]:
dataset.push_to_hub("ctu-aic/ask_library_cs", commit_message="Columns with corrected text using korektor")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ctu-aic/ask_library_cs/commit/df8ccc4930f65e92cfbb11eb56195e545ac6e36a', commit_message='Columns with corrected text using korektor', commit_description='', oid='df8ccc4930f65e92cfbb11eb56195e545ac6e36a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ctu-aic/ask_library_cs', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ctu-aic/ask_library_cs'), pr_revision=None, pr_num=None)

## Other

In [ ]:
import nltk

def filter_correct_diacritics(example):
    edit_distance_q = nltk.edit_distance(example['question'], example['question_diacritics_korektor'])
    edit_distance_a = nltk.edit_distance(example['answer'], example['answer_diacritics_korektor'])
    threshold = len(example['question']) / 2

    return edit_distance_q < threshold and edit_distance_a < threshold

def filter_correct_spell(example):
    return example['question'] == example['question_spell_korektor'] and example['answer'] == example['answer_spell_korektor']


correct_diacritics = dataset.filter(filter_correct_diacritics)
#correct_spell = dataset.filter(filter_correct_spell)

Filter:   0%|          | 0/16273 [00:00<?, ? examples/s]

In [4]:
dataset[100]

{'title': 'Advence a konvence',
 'question': 'Jaký je rozdíl mezi advekcí a konvekcí (týká seatmosféry)?',
 'answer': 'Dobrý den, jako advekce se v meteorologii označuje přemísťování vzduchových hmot daných vlastností ve vodorovném směru nebo přenos určité veličiny nebo fyzikální vlastnosti vzduchu (teploty, tlaku) vodorovným prouděním (větrem) v atmosféře. Konvekce znamená uspořádaný pohyb vzduchu v atmosféře ve vertikálním směru, např. termická konvekce vzniká při nestabilním zvrstvení atmosféry, při kterém teplejší vzduch stoupá vzhůru, ohřívá chladnější vyšší vrstvy atmosféry, vznikají stoupavé vzdušné proudy. Konvekční pohyby mohou mít nespojitý charakter, tj. probíhají ve formě pohybu jednotlivých vzduchových "bublin", nebo nabývají tvaru spojitých vertikálních vzdušných proudů. Konvekce je zpravidla doprovázena turbulencí a spolu s ní se významně podílí na promíchávání vzduchu a přenosu tepla i vodní páry od zemského povrchu do atmosféry. Konvekce se vyskytuje jen v teplejších r

In [4]:
dataset

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language', 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor'],
    num_rows: 16273
})

In [5]:
correct_diacritics[0]["question"] == correct_diacritics[0]["question_diacritics_korektor"]

False

In [6]:
correct_diacritics[0]

{'title': 'e-knihy',
 'question': 'Dobrý den, nemohu se prosím přihlásit přes zadané údaje, hlásí to chybně zadaný pin nebo čtenářské číslo, delší období jsem knihy nevypůjčovala, chci teď v době koronaviru zkusit e-lerning. Děkuji za odpověď.',
 'answer': 'Vážená paní,  moc mě to mrzí, ale v databázi čtenářů Vás již nemáme. Podle Nařízení na ochranu osobních údajů nemůžeme spravovat údaje čtenářů déle než šest měsíců od propadlé registrace. Potom musíme všechny Vaše údaje zarchivovat. Nabídnout Vám mohu založení nového čtenářského konta, toto lze však učinit až v okamžiku znovuotevření knihovny.  Momentálně můžete zkusit e-knihy, které jdou stahovat zdarma, více informací získáte na tomto webu -https://protiviru.knihovny.cz/  Děkuji za pochopení a přeji pevné zdraví',
 'okres': 'Hradec Králové',
 'library': 'Knihovna města Hradce Králové',
 'category': 'Informace o knihovně',
 'used_sources': None,
 'date': Timestamp('2020-03-23 08:00:00'),
 'url': 'https://www.ptejteseknihovny.cz/dot

In [7]:
#what are differences between original and corrected text
def diff_text(example):
    print(f"Original question: {example['question']}")
    print(f"Corrected question: {example['question_diacritics_korektor']}")
    print(f"Original answer: {example['answer']}")
    print(f"Corrected answer: {example['answer_diacritics_korektor']}")
    print()

diff_text(correct_diacritics[0])
print("\sd")

Original question: Dobrý den, nemohu se prosím přihlásit přes zadané údaje, hlásí to chybně zadaný pin nebo čtenářské číslo, delší období jsem knihy nevypůjčovala, chci teď v době koronaviru zkusit e-lerning. Děkuji za odpověď.
Corrected question: Dobrý den, nemohu se prosím přihlásit přes zadané údaje, hlásí to chybně žádaný pin nebo čtenářské číslo, delší období jsem knihy nevypůjčovala, chci teď v době koronaviru zkusit e-lerning. Děkuji za odpověď.
Original answer: Vážená paní,  moc mě to mrzí, ale v databázi čtenářů Vás již nemáme. Podle Nařízení na ochranu osobních údajů nemůžeme spravovat údaje čtenářů déle než šest měsíců od propadlé registrace. Potom musíme všechny Vaše údaje zarchivovat. Nabídnout Vám mohu založení nového čtenářského konta, toto lze však učinit až v okamžiku znovuotevření knihovny.  Momentálně můžete zkusit e-knihy, které jdou stahovat zdarma, více informací získáte na tomto webu -https://protiviru.knihovny.cz/  Děkuji za pochopení a přeji pevné zdraví
Correc

In [8]:
q = correct_diacritics[0]["question"]
for i in range(len(q)):
    if q[i] != correct_diacritics[0]["question_diacritics_korektor"][i]:
        print(q[i], correct_diacritics[0]["question_diacritics_korektor"][i])

z ž
a á


In [31]:
correct_spell

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language', 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor'],
    num_rows: 91
})

In [2]:
#filter czech
dataset = dataset.filter(lambda example: example['language'] == 'cs')
dataset

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language'],
    num_rows: 15789
})

In [3]:
dataset[895]

{'title': 'brát či dávat',
 'question': 'když mě svrbí dlaň říká se ,že budu brát, ale pravánebo levá, brát či dávat',
 'answer': 'Dobrá den, k Vašemu dotazu se nám podařilo dohledat následující informace : u Romů se dodnes traduje několik pověr ve spojení s penězi. V jedné z nich se říká, že když vás svrbí levá dlaň, znamená to, že v nejbližší době obdržíte nějaké peníze. A když vás naopak svrbí pravá dlaň, Romové říkají, že budete muset něco zaplatit. Levá bere, pravá dává. Nejsem pověrčivý, ale říká se, že jsou věci mezi nebem a zemí. U nás tomu věřila pouze matka: když si škrábala levou dlaň, říkala, že už brzy přijde listonoška a přinese jí peníze. My jsme se jen posmívali, ale vždy tomu tak bylo. Nikdy se nestalo, že by ji tato pověra zklamala. V životě jsem se několikrát přesvědčil, že znamení této pověry je pravdivé a dnes už jí věřím i já. Vždy si vzpomenu, jak nám matka vysvětlovala, že každé svrbění levé dlaně není stejné. Podle ní vás začne svrbět levá dlaň vždy dva až tři 

In [4]:
#filter all newline and whitespace characters
import re

def clean_text(example):
    #match all whitespace characters and replace them with a single space
    example['question'] = re.sub(r'\s+', ' ', example['question']).strip()
    example['answer'] = re.sub(r'\s+', ' ', example['answer']).strip()
    return example

dataset = dataset.map(clean_text)

In [27]:
correct_text("Jak se mas?")

<Response [200]>


'Jak se máš?'

In [15]:
FILTER_STRINGS = ["Děkuji za odpověď.", "Prosíme o laskavou odpověď", "Mnohokrát děkuji.", "děkuji", "Děkuji a velice se těším na odpověď."]

#strip the strings!

#diacritics restoration ufal bert/czaccent/korektor ufal/GECCC finetuned ufal

#filter out containing links -> too few -> another link filtering?

#look at the data without these greetings
FILTER_GREETING = ["Vážený paní", "Dobrý den", "Vážený pane", "Vážený čtenáři", "Vážená paní", "Dobré odpoledne"]
dataset_g = dataset.filter(lambda x: not any(greeting in x['answer'] for greeting in FILTER_GREETING) and not any(greeting.lower() in x['answer'] for greeting in FILTER_STRINGS))
dataset_g

Filter:   0%|          | 0/15789 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language'],
    num_rows: 2282
})

In [16]:
dataset_g[202]

{'title': 'ČERNÝ, Jan. Knieha lékarska, kteráž slove herbář aneb Zelinář',
 'question': 'Dobrý den, hledám informace o knize Knieha lekarska Kteraz slowe herbarz, Mikuláše Klaudiána, z roku 1517. Za jakoukoliv zprávu předem děkuji.',
 'answer': 'Dobry den, pravdepodobne mate na mysli herbar Jana Cerneho: CERNY, Jan (Niger). Knieha lekarska kteraz slowe herbarz: aneb zelinarz :welmi vziteczna: z mnohych knieh latinskych. y zskutecznych praczij wybrana:poczina se sstiastnie... Norimberk, 1517. Podrobny bibliograficky zaznam knihy vcetne informaci o knihovnach, ktere herbar ve svych sbirkach vlastni, naleznete v databazi Knihopis Digital -http://db.knihopis.org/l.dll?cll~1758, soucasti zaznamu jsou i nahledy nekterych stranek -http://db.knihopis.org/l.dll?cli~1758&TL=. Pokud byste mel zajem, muzete si napr. v NK CR vypujcit kriticke vydani Kniehy lekarske z roku 1981. Podminkou je platny ctenarsky prukaz NK CR (http://www.nkp.cz/pages/page.php3?nazev=Jak_to_tady_chodi&submenu3=98),knihu s

In [11]:
import re
import nltk
nltk.download('punkt_tab')

def remove_url(text:str) -> str:
    url_in_brackets_pattern = re.compile(r'\s*\([^)(]*https?:\/\/[^\s()]+[^)(]*\)')
    if "http" in text:
        #remove urls in brackets
        text = re.sub(url_in_brackets_pattern, '', text)

        #if url still in the answer
        if "http" in text:
            #handle standalone urls
            ##obtain sentences
            sentences=nltk.tokenize.sent_tokenize(text, language="czech")
            ##find sentences without urls
            filtered_sentences = [sentence for sentence in sentences if "http" not in sentence]
            
            text = " ".join(filtered_sentences)

    
    return text

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/mlynatom/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [18]:
dataset_g["answer"][202]

'Dobry den, pravdepodobne mate na mysli herbar Jana Cerneho: CERNY, Jan (Niger). Knieha lekarska kteraz slowe herbarz: aneb zelinarz :welmi vziteczna: z mnohych knieh latinskych. y zskutecznych praczij wybrana:poczina se sstiastnie... Norimberk, 1517. Podrobny bibliograficky zaznam knihy vcetne informaci o knihovnach, ktere herbar ve svych sbirkach vlastni, naleznete v databazi Knihopis Digital -http://db.knihopis.org/l.dll?cll~1758, soucasti zaznamu jsou i nahledy nekterych stranek -http://db.knihopis.org/l.dll?cli~1758&TL=. Pokud byste mel zajem, muzete si napr. v NK CR vypujcit kriticke vydani Kniehy lekarske z roku 1981. Podminkou je platny ctenarsky prukaz NK CR (http://www.nkp.cz/pages/page.php3?nazev=Jak_to_tady_chodi&submenu3=98),knihu si muzete objednat prostrednictvim Elektronickeho katalogu NK CR - baze NKC (http://sigma.nkp.cz/cze/nkc). Soucasti knihy je uvod, ktery herbar blize charakerizuje. ČERNÝ, Jan. Knieha lékarska, kteráž slove herbář aneb Zelinář. Praha : Avicenum, 

In [19]:
remove_url(dataset_g["answer"][202])

'Dobry den, pravdepodobne mate na mysli herbar Jana Cerneho: CERNY, Jan (Niger). Knieha lekarska kteraz slowe herbarz: aneb zelinarz :welmi vziteczna: z mnohych knieh latinskych. y zskutecznych praczij wybrana:poczina se sstiastnie... Norimberk, 1517. Pokud byste mel zajem, muzete si napr. v NK CR vypujcit kriticke vydani Kniehy lekarske z roku 1981. Podminkou je platny ctenarsky prukaz NK CR,knihu si muzete objednat prostrednictvim Elektronickeho katalogu NK CR - baze NKC. Soucasti knihy je uvod, ktery herbar blize charakerizuje. ČERNÝ, Jan. Knieha lékarska, kteráž slove herbář aneb Zelinář. Praha : Avicenum, 1981. 463 s. Resume: Prvni kriticke vydani zakladniho spisu ceskeho humanistickeho lekare, ktery poprve vysel v Norimberce r. 1517. Cerneho cesky psany herbar se vyznacuje kritickym pristupme k dane problematice a snahou o usporadani tehdejsiho chaotickeho botanickeho nazvoslovi. Cerny dava kazde rostline jediny cesky nazev a podrobnym popisem umoznuje jeji presne urceni. Herbar 